In [1]:
import copy
import random
import time
import matplotlib.pyplot as plt
from dataclasses import dataclass
from enum import Enum

@dataclass
class Node:
    student_info: list
    calendar: list

class Student(Enum):
    MARIA = 0
    GUS = 1
    DIEGO = 2
    JOSE = 3

class Subject(Enum):
    MATH = 0
    HISTORY = 1
    PHYSICS = 2
    CHEMISTRY = 3
    SPANISH = 4
    ENGLISH = 5
    BIOLOGY = 6

class Day(Enum):
    MONDAY = 0
    TUESDAY = 1
    WEDNESDAY = 2

MAX_CLASSES_PER_STUDENT = 3
MAX_CLASSES_PER_DAY = len(Student)

def initial_node_initializer():
    student_info = {}

    for student in Student:
        classes = pick_subjects()
        days = random.sample(list(Day), MAX_CLASSES_PER_STUDENT)
        student_info[student] = list(zip(classes, days))

    calendar = do_calendar(student_info)

    return Node(student_info, calendar)

def pick_subjects():
    global global_subjects

    if len(global_subjects) == 0:
        global_subjects = list(Subject)

    if len(global_subjects) < MAX_CLASSES_PER_STUDENT:
        classes = [global_subjects.pop() for _ in range(len(global_subjects))]
        len_classes = len(classes)
        global_subjects = list(Subject)
        random.shuffle(global_subjects)

        for _ in range(MAX_CLASSES_PER_STUDENT - len_classes):
            classes.append(global_subjects.pop())
    else:
        random.shuffle(global_subjects)
        classes = [global_subjects.pop() for _ in range(MAX_CLASSES_PER_STUDENT)]

    return classes

def do_calendar(student_info):
    calendar = {day: [] for day in Day}

    for student in Student:
        for subject, day in student_info[student]:
            calendar[day].append(subject)

    return calendar

def scoring_function(node: Node):
    calendar = node.calendar
    student_info = node.student_info
    repeated_subjects_count = 0

    for day in calendar.keys():
        days_set = set(calendar[day])
        if len(days_set) < MAX_CLASSES_PER_DAY:
            repeated_subjects_count +=  MAX_CLASSES_PER_DAY - len(days_set)

    for student in student_info.keys():
        days_set = set(day for _, day in student_info[student])
        if len(days_set) < MAX_CLASSES_PER_STUDENT:
            repeated_subjects_count += MAX_CLASSES_PER_STUDENT - len(days_set)

    return repeated_subjects_count

def local_search(initial_node):
    current_node = initial_node
    current_score = scoring_function(current_node)

    while True:
        neighbors = generate_neighbors(current_node)
        best_neighbor = min(neighbors, key=lambda node: scoring_function(node))
        best_neighbor_score = scoring_function(best_neighbor)

        if best_neighbor_score >= current_score:
            return current_node
        else:
            current_node = best_neighbor
            current_score = best_neighbor_score

def generate_neighbors(node):
    neighbors = []
    student_info = node.student_info

    for student in student_info.keys():
        for i, (subject, day) in enumerate(student_info[student]):
            for new_day in Day:
                if new_day != day:
                    new_student_info = copy.deepcopy(student_info)
                    new_student_info[student][i] = (subject, new_day)
                    neighbors.append(Node(new_student_info, do_calendar(new_student_info)))

    return neighbors

# Testing the local search
global_subjects = list(Subject)
init = initial_node_initializer()

start_time = time.time()
result = local_search(init)
end_time = time.time()

print("Solution found:")
for student, subjects in result.student_info.items():
    print(f"{student.name}:")
    for subject, day in subjects:
        print(f"  {subject.name} on {day.name}")
    print()

print("Execution time:", end_time - start_time)


Solution found:
MARIA:
  HISTORY on WEDNESDAY
  CHEMISTRY on MONDAY
  ENGLISH on TUESDAY

GUS:
  PHYSICS on TUESDAY
  SPANISH on MONDAY
  MATH on WEDNESDAY

DIEGO:
  BIOLOGY on TUESDAY
  HISTORY on WEDNESDAY
  CHEMISTRY on MONDAY

JOSE:
  BIOLOGY on WEDNESDAY
  ENGLISH on MONDAY
  PHYSICS on TUESDAY

Execution time: 0.003535747528076172
